In [ ]:
import sys
import os
import random
import re
import pandas as pd
from tqdm import tqdm
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Colab-specific setup
    !pip install langchain langchain_ollama pandas tqdm
    !git clone https://github.com/ostinsolo/Audio-Engineer-Sound-Design-LLM.git
    %cd Audio-Engineer-Sound-Design-LLM/ai_gen
else:
    # Local system setup
    # Check if we're in the correct directory, if not, change to it
    if not os.path.exists('ableton_data.py'):
        os.chdir('/Users/ostinsolo/Documents/Code/Audio-Engineer-Sound-Design-LLM/ai_gen')
    
    # Create and activate a virtual environment named 'aigen'
    !python -m venv aigen
    if sys.platform == "win32":
        !aigen\Scripts\activate
    else:
        !source aigen/bin/activate
    
    # Install required packages in the virtual environment
    !pip install langchain langchain_ollama pandas tqdm

# Add the current directory to Python's path
sys.path.append('.')

# Load the Ableton data
from ableton_data import *

# Load the Ollama LLM
llm = OllamaLLM(model="llama3.2")

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["ableton_data"],
    template="""
Generate a unique utterance for an Ableton Live task and its corresponding action order. Use the provided Ableton data to ensure relevance and accuracy.

Ableton Data:
{ableton_data}

Format the output as follows:
Utterance: [insert utterance here]
Action Order: ["step 1", "step 2", ...]

Rules:
1. Track creation actions don't include a track number (the track doesn't exist yet).
2. Other track-related actions always start with "track {ableton_data[track_number]}".
3. Device-related actions always start with "track {ableton_data[track_number]}", "search device", followed by the device name.
4. For instruments, the order is: "track {ableton_data[track_number]}", "search device", "{ableton_data[instrument]}", "{ableton_data[device_type]}".
5. Audio effects are treated separately from instruments.
6. Value actions include the device name, parameter, action (set/increase/decrease), speed modifier, and value.
7. Project actions are global and don't require a track number.
8. Clip actions always include a track number and clip number.
9. View actions change the current view in Ableton Live and don't require a track number.
10. Mapping actions involve assigning controls to parameters and may not require a track number.

Placeholder Usage:
- Use {ableton_data[track_creation_actions]} for creating new tracks.
- Use {ableton_data[track_actions]} for actions on specific tracks.
- Use {ableton_data[audio_effect]} for audio effects.
- Use {ableton_data[instrument]} and {ableton_data[device_type]} for instruments.
- Use {ableton_data[project_actions]} for global project actions.
- Use {ableton_data[track_type]} when specifying track types.
- Use {ableton_data[clip_actions]} for actions on clips.
- Use {ableton_data[device_name]} and {ableton_data[parameter]} when adjusting devices.
- Use {ableton_data[value_actions]} and {ableton_data[speed_modifiers]} for value changes.
- Use {ableton_data[view_actions]} for changing views in Ableton Live.
- Use {ableton_data[mapping_actions]} for control mapping operations.

Action Types:
1. Track creation actions: Create new tracks without a track number.
2. Track actions: Perform on specific tracks, require a track number.
3. Project actions: Global actions, no track number required.
4. Device actions: Perform on devices within tracks, require track number and device name.
5. Clip actions: Perform on clips within tracks, require track number and clip number.
6. Value actions: Modify device parameters, require track number, device name, parameter, action, speed modifier, and value.
7. View actions: Change the current view, no track number required.
8. Mapping actions: Assign controls to parameters, may or may not require a track number.

Now generate a new, unique utterance and action order:
"""
)

def extract_utterance_and_action(response):
    utterance_match = re.search(r'Utterance: (.+)', response)
    action_order_match = re.search(r'Action Order: (\[.+\])', response)
    
    utterance = utterance_match.group(1) if utterance_match else None
    action_order = eval(action_order_match.group(1)) if action_order_match else None
    
    return utterance, action_order

def generate_utterance_and_action():
    # Randomly select elements from the Ableton data to encourage variety
    audio_effect = random.choice(audio_effects)
    instrument = random.choice(list(device_types.keys()))
    device_type = random.choice(device_types[instrument])
    
    # Correctly handle actions and speed modifiers
    track_creation_action = random.choice(actions['track_creation_actions'])
    track_action = random.choice(actions['track_actions'])
    project_action = random.choice(actions['project_actions'])
    clip_action = random.choice(actions['clip_actions'])
    value_action = random.choice(actions['value_actions'])
    view_action = random.choice(actions['view_actions'])
    mapping_action = random.choice(actions['mapping_actions'])
    
    speed_modifier_category = random.choice(list(actions['speed_modifiers'].keys()))
    speed_modifier = random.choice(actions['speed_modifiers'][speed_modifier_category])
    
    template = random.choice(utterance_templates)
    
    # Generate random track numbers
    track_number = random.randint(1, 8)
    track_number1 = random.randint(1, 8)
    track_number2 = random.randint(1, 8)
    while track_number2 == track_number1:
        track_number2 = random.randint(1, 8)
    
    # Generate other random values
    clip_number = random.randint(1, 16)
    value = random.randint(0, 100)
    map_number = random.randint(1, 128)
    
    # Create a simplified version of the Ableton data to pass to the AI
    simplified_data = {
        'track_number': track_number,
        'track_number1': track_number1,
        'track_number2': track_number2,
        'clip_number': clip_number,
        'value': value,
        'number': map_number,
        'audio_effect': audio_effect,
        'instrument': instrument,
        'device_type': device_type,
        'track_creation_actions': track_creation_action,
        'track_actions': track_action,
        'project_actions': project_action,
        'clip_actions': clip_action,
        'value_actions': value_action,
        'view_actions': view_action,
        'mapping_actions': mapping_action,
        'speed_modifiers': speed_modifier,
        'track_type': random.choice(track_types),
        'device_name': random.choice(audio_effects + list(device_types.keys())),
        'parameter': random.choice(parameters)
    }
    
    # Format the template with the simplified data
    try:
        formatted_template = template.format(**simplified_data)
    except KeyError as e:
        print(f"KeyError: {e} is missing in the simplified_data")
        print("Template:", template)
        print("Simplified data:", simplified_data)
        return None, None
    
    # Pass the simplified_data to the prompt_template, not the formatted_template
    prompt = prompt_template.format(ableton_data=simplified_data)
    output = llm(prompt)
    return extract_utterance_and_action(output)

# Generate the data
num_generations = 100  # You can adjust this number
generated_data = []

for _ in tqdm(range(num_generations), desc="Generating utterances and actions"):
    utterance, action_order = generate_utterance_and_action()
    if utterance and action_order:
        generated_data.append({"Utterance": utterance, "Action_Order": action_order})

print(f"Generated {len(generated_data)} utterances and action orders.")

In [ ]:
# Enable tqdm for pandas operations
tqdm.pandas()

def create_csv_from_ai_output(input_data, output_file):
    # Convert the input data to a pandas DataFrame
    df = pd.DataFrame(input_data)
    
    # Convert the Action_Order list to a comma-separated string
    df['Action_Order'] = df['Action_Order'].progress_apply(lambda x: ', '.join(x))
    
    # Write the DataFrame to a CSV file
    df.to_csv(output_file, index=False)
    print(f"CSV file '{output_file}' has been created successfully.")

# Create the CSV file
output_file = 'ableton_utterances_and_actions.csv'
create_csv_from_ai_output(generated_data, output_file)